In [1]:
import pygrib
import numpy as np
from pprint import pprint
import torch

In [2]:
filename = "/mnt/pami/DATASET/METEOROLOGY/ECforecast/GRIB/2016/101912/C1D10191200101918001"

In [3]:
myfile = pygrib.open(filename)

In [4]:
for data in myfile:
    print(data)

1:Specific cloud ice water content:kg kg**-1 (instant):regular_ll:hybrid:level 93:fcst time 6 hrs:from 201610191200
2:Fraction of cloud cover:(0 - 1) (instant):regular_ll:hybrid:level 93:fcst time 6 hrs:from 201610191200
3:Specific cloud liquid water content:kg kg**-1 (instant):regular_ll:hybrid:level 102:fcst time 6 hrs:from 201610191200
4:Potential vorticity:K m**2 kg**-1 s**-1 (instant):regular_ll:isobaricInhPa:level 20:fcst time 6 hrs:from 201610191200
5:Temperature:K (instant):regular_ll:isobaricInhPa:level 100:fcst time 6 hrs:from 201610191200
6:Specific humidity:kg kg**-1 (instant):regular_ll:isobaricInhPa:level 800:fcst time 6 hrs:from 201610191200
7:Specific humidity:kg kg**-1 (instant):regular_ll:isobaricInhPa:level 925:fcst time 6 hrs:from 201610191200
8:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 6 hrs:from 201610191200
9:Logarithm of surface pressure:~ (instant):regular_ll:hybrid:level 1:fcst time 6 hrs:from 201610191200
10:Specific cloud liquid wa

In [5]:
total = myfile.select(name="Total precipitation")[0]
convective = myfile.select(name="Convective precipitation")[0]
data, lats, lons = total.data(lat1=28,lat2=36,lon1=112,lon2=124)
print(data)


[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.56127930e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [6.10351562e-05 6.10351562e-05 1.44958496e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.62939453e-05 1.60217285e-04 3.35693359e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.44958496e-04 4.34875488e-04 6.86645508e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [6]:
print(lats)
print(lons)

[[36.    36.    36.    ... 36.    36.    36.   ]
 [35.875 35.875 35.875 ... 35.875 35.875 35.875]
 [35.75  35.75  35.75  ... 35.75  35.75  35.75 ]
 ...
 [28.25  28.25  28.25  ... 28.25  28.25  28.25 ]
 [28.125 28.125 28.125 ... 28.125 28.125 28.125]
 [28.    28.    28.    ... 28.    28.    28.   ]]
[[112.    112.125 112.25  ... 123.75  123.875 124.   ]
 [112.    112.125 112.25  ... 123.75  123.875 124.   ]
 [112.    112.125 112.25  ... 123.75  123.875 124.   ]
 ...
 [112.    112.125 112.25  ... 123.75  123.875 124.   ]
 [112.    112.125 112.25  ... 123.75  123.875 124.   ]
 [112.    112.125 112.25  ... 123.75  123.875 124.   ]]


In [14]:
data_mask = data>0.1
print(data_mask)
res_data = data*data_mask
print(res_data)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
import os
from pprint import pprint

import math
import torch
def grib_bilinear_interpolation(grib, grib_lats, grib_lons, gt_lat, gt_lon):
    lon_idx_0, lon_idx_1 = binary_search(grib_lons, gt_lon, False)
    lat_idx_0, lat_idx_1 = binary_search(grib_lats, gt_lat, True)
    x_0 = grib_lons[lon_idx_0]
    x_1 = grib_lons[lon_idx_1]
    y_0 = grib_lats[lat_idx_0]
    y_1 = grib_lats[lat_idx_1]

    w_0 = grib[lat_idx_1, lon_idx_0] * (x_1 - gt_lon) / (x_1 - x_0 + 1e-5) + grib[lat_idx_1, lon_idx_1] * (gt_lon - x_0) / (
        x_1 - x_0 + 1e-5)
    w_1 = grib[lat_idx_0, lon_idx_0] * (x_1 - gt_lon) / (x_1 - x_0 + 1e-5) + grib[lat_idx_0, lon_idx_1] * (gt_lon - x_0) / (
        x_1 - x_0 + 1e-5)

    res = w_0 * (y_0 - gt_lat) / (y_0 - y_1 + 1e-5) + w_1 * (gt_lat - y_1) / (y_0 - y_1 + 1e-5)
    return res

"""
二分查找，返回离观测站最近的两个网格坐标点的索引
"""
def binary_search(input_array, item, is_lat):
    if is_lat:
        input_array = input_array[::-1]
    

    low = 0
    high = len(input_array) - 1
#     print(len(input_array))
#     pr

    while low <= high:
        mid = math.floor((low + high) / 2)
        guess = input_array[mid]
        if guess == item:
            if (is_lat):
                return len(input_array) - mid - 1, len(input_array) - mid - 1
            else:
                return mid, mid
        elif guess > item:
            high = mid - 1
        else:
            low = mid + 1
    if (is_lat):
        return len(input_array) - low - 1, len(input_array) - high - 1
    else:
        return high, low
# micaps_filename = "/mnt/pami/DATASET/METEOROLOGY/GT_micaps/2016/07/surface/r6-p"
# filename_list = sorted(os.listdir(micaps_filename))
# item = filename_list[20]
# pprint(filename_list)
filename = "/mnt/pami/DATASET/METEOROLOGY/GT_micaps/2016/10/surface/r6-p/16101911.000"
# filename = os.path.join(micaps_filename,item)
with open(filename,encoding="GBK") as f:
    data = f.readlines()
    micaps_list = data[14:]
    for item in micaps_list:
        micaps_data = item.split()
        lon = float(micaps_data[1])
        lat = float(micaps_data[2])
        values = float(micaps_data[3])
        target = (lat,lon,values)        
        if (float(lat) < 29) or (float(lat) > 35):
            continue
        elif (float(lon) < 113) or (float(lon) > 123):
            continue
        else:
            pprint(micaps_data)
            print(res)
            
            p_value = grib_bilinear_interpolation(torch.Tensor(res),lats,lons,
                                                  lat,lon)
            pprint(p_value)

In [ ]:
pprint(convective.values)

In [ ]:
import os
import datetime
dat_filename = "/mnt/pami/DATASET/METEOROLOGY/16-17typhoon"
filename_list = sorted(os.listdir(dat_filename))
for item in filename_list:
    filename = os.path.join(dat_filename,item)
    with open(filename,encoding="GBK") as f:
        data = f.readlines()
        for item in data:
            line = item.split()
            month = line[8]
            day = line[9]
            hour = line[10]
            lon = line[12]
            lat = line[13]
            print(item)

In [10]:
import os
from pprint import pprint
micaps_filename = "/mnt/pami/DATASET/METEOROLOGY/GT_micaps/2017/06/surface/r6-p/"
filename_list = sorted(os.listdir(micaps_filename))
item = filename_list[21]
filename = os.path.join(micaps_filename,item)
with open(filename,encoding="GBK") as f:
    data = f.readlines()
#     pprint(len(data))
#     pprint(type(data))
    micaps_list = data[14:]
    for items in micaps_list:
        micaps_data = items.split()
        lat = micaps_data[1]
        lon = micaps_data[2]
        values = micaps_data[4]
        pprint(micaps_data)
#         pprint(item.split())
#     pprint(micaps_lis
    

['57574', '112.40', '29.37', '36', '0']
['54777', '122.07', '37.20', '118', '0']
['50879', '130.55', '46.22', '187', '0']
['58016', '116.97', '34.18', '35', '4.0']
['58107', '115.28', '33.02', '36', '0']
['58230', '118.28', '32.12', '27', '0']
['58426', '118.13', '30.30', '193', '0']
['58441', '119.42', '30.87', '43', '0']
['58627', '117.03', '28.25', '56', '0']
['58601', '115.38', '28.70', '54', '0.1']
['57505', '105.58', '29.42', '338', '0.01']
['57409', '105.80', '30.18', '298', '0']
['57525', '107.75', '29.32', '278', '0.4']
['57348', '109.53', '31.02', '300', '0']
['57431', '108.35', '30.83', '257', '1.0']
['57633', '108.77', '28.82', '827', '0']
['57502', '105.70', '29.70', '395', '0']
['57510', '106.07', '29.85', '326', '0']
['57518', '106.50', '29.33', '506', '0']
['57522', '107.27', '29.73', '373', '0.5']
['57536', '108.78', '29.53', '607', '3.0']
['57513', '106.62', '29.73', '465', '0']
['57426', '107.80', '30.68', '455', '4.0']
['57517', '106.25', '29.28', '261', '0']
['5751

['56196', '104.73', '31.45', '523', '0.4']
['57306', '105.97', '31.58', '383', '0']
['56182', '103.60', '32.67', '2881', '0.2']
['56462', '101.50', '29.00', '2925', '0']
['53759', '110.83', '36.98', '1109', '0']
['53678', '112.65', '38.07', '897', '0']
['53771', '112.05', '37.42', '750', '0']
['53788', '113.57', '37.33', '1266', '0']
['53772', '112.58', '37.62', '776', '0']
['53873', '112.87', '36.10', '951', '0']
['53964', '111.22', '35.62', '431', '0']
['53663', '111.82', '38.92', '1401', '0']
['53673', '112.72', '38.73', '828', '0']
['53685', '113.42', '38.07', '953', '0']
['53770', '112.35', '37.35', '769', '0']
['53582', '113.67', '39.72', '1079', '0']
['53967', '111.20', '35.33', '468', '0']
['57060', '111.23', '35.17', '403', '0']
['53963', '111.37', '35.65', '434', '0']
['53957', '110.72', '35.62', '459', '0']
['53588', '113.52', '38.95', '2208', '0']
['53478', '112.45', '40.00', '1346', '0']
['53864', '111.10', '36.40', '1031', '0']
['53875', '112.35', '36.52', '1000', '0']
['

In [ ]:
import os

for root, dirs, files in os.walk("/mnt/pami/DATASET/METEOROLOGY/ECforecast/GRIB/"):
    pprint(root)
    pprint(dirs)
    pprint(files)
    break
    
years = os.listdir("/mnt/pami/DATASET/METEOROLOGY/ECforecast/GRIB/")
dates = [os.listdir(os.path.join(root,year)) for year in years]
pprint(dates)

In [ ]:
import os
testfilestr = "/mnt/pami/DATASET/METEOROLOGY/ECforecast/GRIB/2016/073100/C1D07310000073118001"
# print(os.path.split(testfilestr))
tmp = testfilestr.split("/")

In [ ]:
print(tmp[-1])
print(tmp[-2])
print(tmp[-3])
print(tmp[-2][:2])
print(os.path.join(tmp[-3],tmp[-2][:2],"surface","r6-p"))

In [ ]:
print(tmp[-1][-5:-3])
print(tmp[-1][-9:-5])

In [ ]:
print(tmp[-3][-2:])



In [ ]:
import math
import torch
import numpy as np
import torch.nn.functional as F

def grib_bilinear_interpolation(grib, grib_lats, grib_lons, gt_lat, gt_lon):
    lon_idx_0, lon_idx_1 = binary_search(grib_lons, gt_lon, False)
    lat_idx_0, lat_idx_1 = binary_search(grib_lats, gt_lat, True)
    print(lon_idx_0, lon_idx_1)
    print(lat_idx_0, lat_idx_1)
    x_0 = grib_lons[lon_idx_0]
    x_1 = grib_lons[lon_idx_1]
    y_0 = grib_lats[lat_idx_0]
    y_1 = grib_lats[lat_idx_1]

    w_0 = grib[:,:,lat_idx_1, lon_idx_0] * (x_1 - gt_lon) / (x_1 - x_0 + 1e-6) + grib[:,:,lat_idx_1, lon_idx_1] * (gt_lon - x_0 ) / (
        x_1 - x_0 + 1e-6)
    w_1 = grib[:,:,lat_idx_0, lon_idx_0] * (x_1 - gt_lon) / (x_1 - x_0 + 1e-6) + grib[:,:,lat_idx_0, lon_idx_1] * (gt_lon - x_0 ) / (
        x_1 - x_0 + 1e-6)

    res = w_0 * (y_0 - gt_lat) / (y_0 - y_1 + 1e-6 ) + w_1 * (gt_lat - y_1) / (y_0 - y_1 + 1e-6)
    return res

"""
二分查找，返回离观测站最近的两个网格坐标点的索引
"""
def binary_search(input_arr, item, is_lat):
    if is_lat:
        input_arr=input_arr[::-1]
    print(input_arr)

    low = 0
    high = len(input_arr) - 1
    print(low,high)
    while low <= high:
        mid = math.floor((low + high) / 2)
        guess = input_arr[mid]
        if guess == item:
            if is_lat:
                return len(input_arr) - mid - 1, len(input_arr) - mid - 1
            else:
                return mid, mid
        elif guess > item:
            high = mid - 1
        else:
            low = mid + 1
        print(low,high)

    if is_lat:
        return len(input_arr) - low - 1, len(input_arr) - high - 1
    else:
        return high, low

image = torch.zeros(1, 1, 2, 2)
image[0, :, 1, 1] = 1.
print(image)

output = grib_bilinear_interpolation(image,np.asarray([1,0]),np.asarray([0,1]),0.5,0.2)

# output =F.interpolate(image[:,:,0:1,0:1],size=[1,1],mode="bilinear") 
print(output)
print(output.shape)
print(output.view(-1))

In [34]:
import torch
import torch.nn as nn


data = torch.rand((3,1,10,10))
target = torch.ones((3,1,14,14))


class Residual(nn.Module):
    def __init__(self):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3,stride=1,padding=2)
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=1,kernel_size=3,stride=1,padding=2)
        self.relu = nn.ReLU()
        self.pooling = nn.MaxPool2d(kernel_size=3,stride=1,padding=1)

    def forward(self,x):
#         residual = x
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pooling(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pooling(x)
#         x = x + residual
#         out = self.relu(x)
#         print("conv1 param: \n",self.conv1.weight)
#         print("conv2 param: \n",self.conv2.weight)
        return x
    
model = Residual()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

for i in range(1,2):
    optimizer.zero_grad()
    
    output = model(data)
    
    loss = nn.MSELoss()(output,target)
    print(loss.item())
    print(loss.sum().item())
    
    loss.backward()
    
#     for param in model.parameters():
#         print("{} th param grad {}".format(i,param.grad.data.numpy()))
    
    optimizer.step()


0.6019853949546814
0.6019853949546814
